In [1]:
import pandas as pd
import numpy as np
import beam_search_module as bsm 
import ast
from plot_sg_and_pop import *
from datetime import datetime, timedelta

import json

In [2]:
def convert_timestamps(timestamp_str):
    # Remove the square brackets and split the string
    timestamps = timestamp_str.strip('[]').split(', ')
    print(timestamps)
    # Extract the actual datetime strings and convert to datetime objects
    return [pd.Timestamp(ts.split('(')[-1].split(')')[0]) for ts in timestamps]


In [3]:
stock_df = pd.read_csv('data/stock_data_all_features.csv')
stock_df['growth_target'] = stock_df['growth_target'].apply(ast.literal_eval)
stock_df['target'] = stock_df['target'].apply(ast.literal_eval)
stock_df['target'] = stock_df['target'].apply(lambda x: [((i / x[0])-1)*100 for i in x])
stock_df_all = stock_df.copy()
stock_df = stock_df_all.drop(['target', 'dates', 'symbol', 'growth_mean', 'growth_median', 'growth_std_dev'], axis=1)
stock_df_all


,symbol,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,...,growth_best_day_of_week,growth_best_month,growth_best_quarter,growth_best_day_of_month,growth_best_week_of_month,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc,growth_biggest_continuous_increase_perc_start,growth_biggest_continuous_decrease_perc_start,growth_target
0,AOF.DE,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,...,3,8,3,26,34,16.82441,-17.72197,78.0,242.0,"[0.0, 1.02, 0.1, -1.31, 2.49, -0.94, 3.71, -0...."
1,LFG.MI,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,...,1,1,1,2,5,18.59324,-8.00000,54.0,58.0,"[0.0, -2.03, 0.0, 2.07, -2.03, 3.45, -2.0, -2...."
2,ENV.MI,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,...,2,9,3,28,37,11.23512,-6.57895,37.0,19.0,"[0.0, -3.33, -0.69, 4.17, -0.67, -1.34, -1.36,..."
3,M07.F,United States,Security & Protection Services,EUR,Europe/Berlin,FRA,Industrials,0.0,14.054,5.451780e+09,...,4,4,2,28,15,9.97164,-10.13175,18.0,220.0,"[0.0, -0.67, -0.68, 1.36, 0.0, -1.34, 0.0, 0.0..."
4,MUX.DE,Germany,Asset Management,EUR,Europe/Berlin,GER,Financial Services,11244.0,-2.942,3.008613e+08,...,0,5,2,29,19,14.24548,-26.26091,129.0,257.0,"[0.0, -2.56, -0.75, -0.38, 4.17, -1.64, 2.59, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7844,INRN.SW,Switzerland,Specialty Industrial Machinery,CHF,Europe/Zurich,EBS,Industrials,1124.0,15.530,1.979479e+09,...,4,4,2,15,13,17.62885,-12.85629,50.0,219.0,"[0.0, -3.21, 0.83, -0.41, 0.62, -1.43, 2.49, 1..."
7845,RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,...,4,9,3,10,39,13.67960,-9.63956,245.0,54.0,"[0.0, 0.48, 0.0, -4.75, 0.0, 0.0, 2.97, 0.0, 0..."
7846,KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,...,4,9,3,26,39,9.76675,-8.39559,167.0,41.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.51, 0.0, 2.3..."
7847,MAW.L,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,...,4,9,3,26,39,14.62789,-10.74116,55.0,115.0,"[0.0, 0.0, 0.0, 0.0, 2.95, 0.0, 0.0, 0.0, 0.0,..."


In [4]:
# Initialize start and end dates
start_date = datetime(2023, 10, 2)  # Monday, 2 October 2023
end_date = datetime(2024, 9, 29)    # Sunday, 29 September 2024

# Create a list of business days (Monday to Friday)
current_date = start_date
dates = []

# Loop through dates, adding only weekdays (Mon-Fri)
while current_date <= end_date:
    if current_date.weekday() < 5:  # 0 = Monday, 1 = Tuesday, ..., 4 = Friday
        dates.append(current_date)
    current_date += timedelta(days=1)

In [5]:
domain_attributes = True
time_series_attributes = True

if not domain_attributes and time_series_attributes:
    stock_df = stock_df.iloc[:, 11:]

if domain_attributes and not time_series_attributes:
    domain_features = stock_df.columns[:11]
    columns_keep = list(domain_features) + ['growth_target']
    stock_df = stock_df[columns_keep]

if domain_attributes and time_series_attributes:
    stock_df = stock_df
stock_df

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,...,growth_best_day_of_week,growth_best_month,growth_best_quarter,growth_best_day_of_month,growth_best_week_of_month,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc,growth_biggest_continuous_increase_perc_start,growth_biggest_continuous_decrease_perc_start,growth_target
0,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,...,3,8,3,26,34,16.82441,-17.72197,78.0,242.0,"[0.0, 1.02, 0.1, -1.31, 2.49, -0.94, 3.71, -0...."
1,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,...,1,1,1,2,5,18.59324,-8.00000,54.0,58.0,"[0.0, -2.03, 0.0, 2.07, -2.03, 3.45, -2.0, -2...."
2,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,...,2,9,3,28,37,11.23512,-6.57895,37.0,19.0,"[0.0, -3.33, -0.69, 4.17, -0.67, -1.34, -1.36,..."
3,United States,Security & Protection Services,EUR,Europe/Berlin,FRA,Industrials,0.0,14.054,5.451780e+09,71.777,...,4,4,2,28,15,9.97164,-10.13175,18.0,220.0,"[0.0, -0.67, -0.68, 1.36, 0.0, -1.34, 0.0, 0.0..."
4,Germany,Asset Management,EUR,Europe/Berlin,GER,Financial Services,11244.0,-2.942,3.008613e+08,37.175,...,0,5,2,29,19,14.24548,-26.26091,129.0,257.0,"[0.0, -2.56, -0.75, -0.38, 4.17, -1.64, 2.59, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7844,Switzerland,Specialty Industrial Machinery,CHF,Europe/Zurich,EBS,Industrials,1124.0,15.530,1.979479e+09,15.043,...,4,4,2,15,13,17.62885,-12.85629,50.0,219.0,"[0.0, -3.21, 0.83, -0.41, 0.62, -1.43, 2.49, 1..."
7845,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,...,4,9,3,10,39,13.67960,-9.63956,245.0,54.0,"[0.0, 0.48, 0.0, -4.75, 0.0, 0.0, 2.97, 0.0, 0..."
7846,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,...,4,9,3,26,39,9.76675,-8.39559,167.0,41.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.51, 0.0, 2.3..."
7847,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,1.943,...,4,9,3,26,39,14.62789,-10.74116,55.0,115.0,"[0.0, 0.0, 0.0, 0.0, 2.95, 0.0, 0.0, 0.0, 0.0,..."


In [6]:
column_names = list(stock_df.columns)
beam_width = 20
beam_depth = 3
nr_bins = 8
nr_saved = 10
subgroup_size = len(stock_df) * 0.05
target = 'growth_target'
window_size = 20
target_ind = column_names.index(target)
stock_df['growth_target'] = stock_df['growth_target'].apply(lambda x: x[:len(x) - (len(x) % window_size)])
stock_df_all['growth_target'] = stock_df_all['growth_target'].apply(lambda x: x[:len(x) - (len(x) % window_size)])
print('length growth_target: ',len(stock_df.loc[0, 'growth_target']))

length growth_target:  260


In [7]:
data = stock_df.values.tolist()
all_time_series = [i[target_ind] for i in data]
all_time_series = np.array(all_time_series)
targets_baseline = np.mean(all_time_series, axis=0)
att_indices = list(range(0, len(column_names)))
att_indices.remove(target_ind)
att_columns = [column_names[i] for i in att_indices]
types = bsm.categorize_columns_in_order(stock_df, att_columns)
min_quality_improv = 0.1
agg_func = np.mean

In [8]:
results = bsm.beam_search_with_constraint(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved,
                      subgroup_size, target, types, window_size, min_quality_improv, agg_func)


In [9]:
for r in results:
    print(r)
    print('\n')

(0.2878355489661572, (('growth_mean', -0.06624, <function leeq at 0x0000024C944C15A0>), ('growth_std_dev', 3.40254, <function leeq at 0x0000024C944C15A0>)), 737, 11)


(0.2590480489075242, (('growth_mean', -0.00821, <function leeq at 0x0000024C944C15A0>),), 1963, 11)


(0.25734753585793774, (('growth_std_dev', 1.26949, <function leeq at 0x0000024C944C15A0>),), 982, 2)


(0.25569517217132115, (('og_window_min', 233.0, <function gt at 0x0000024C944C1510>), ('growth_mean', -0.07545, <function leeq at 0x0000024C944C15A0>), ('growth_std_dev', 4.33792, <function leeq at 0x0000024C944C15A0>)), 428, 11)


(0.25501692118782765, (('growth_mean', -0.06624, <function leeq at 0x0000024C944C15A0>),), 982, 11)


(0.25313677947937613, (('og_window_max', 36.0, <function leeq at 0x0000024C944C15A0>), ('og_window_min', 182.0, <function gt at 0x0000024C944C1510>), ('growth_std_dev', 3.10721, <function leeq at 0x0000024C944C15A0>)), 462, 1)


(0.25101468895445456, (('og_window_min', 196.0, <function gt at 

In [6]:
import math
print(stock_df_all.columns)

Index(['symbol', 'country', 'industry', 'currency', 'exchangeTimezoneName',
       'exchange', 'sector', 'averageVolume10days', 'enterpriseToEbitda',
       'marketCap', 'debtToEquity', 'fullTimeEmployees', 'dates', 'target',
       'og_mean', 'og_median', 'og_std_dev', 'og_autocorr_lag1', 'og_min',
       'og_max', 'og_range', 'og_window_min', 'og_window_max',
       'og_trend_slope', 'og_longest_continuous_increase',
       'og_biggest_continuous_increase', 'og_longest_continuous_decrease',
       'og_biggest_continuous_decrease',
       'og_longest_continuous_increase_start',
       'og_biggest_continuous_increase_start',
       'og_longest_continuous_decrease_start',
       'og_biggest_continuous_decrease_start', 'growth_mean', 'growth_median',
       'growth_std_dev', 'growth_autocorr_lag1', 'growth_min', 'growth_max',
       'growth_range', 'growth_window_min', 'growth_window_max',
       'growth_trend_slope', 'growth_best_day_of_week', 'growth_best_month',
       'growth_best_qu

In [69]:
vl = stock_df_all['growth_biggest_continuous_decrease_perc']
vl=sorted(vl)

In [70]:
b=5
split_points = [vl[math.floor(j * ((len(vl))/b))] for j in range(1, b)]
split_points

[-20.71632, -14.87729, -11.33433, -8.4317]

In [72]:
vl[7000:]

[-7.02991,
 -7.02398,
 -7.02367,
 -7.02269,
 -7.02248,
 -7.02093,
 -7.02066,
 -7.02037,
 -7.01844,
 -7.01759,
 -7.01576,
 -7.01346,
 -7.00935,
 -7.00237,
 -7.00036,
 -6.99943,
 -6.99798,
 -6.99675,
 -6.99668,
 -6.99398,
 -6.98829,
 -6.97943,
 -6.97933,
 -6.97674,
 -6.97646,
 -6.97369,
 -6.97302,
 -6.97257,
 -6.97056,
 -6.96673,
 -6.9626,
 -6.96051,
 -6.95811,
 -6.95465,
 -6.95427,
 -6.95067,
 -6.95009,
 -6.94999,
 -6.94904,
 -6.94735,
 -6.93641,
 -6.93545,
 -6.93365,
 -6.93069,
 -6.93019,
 -6.9297,
 -6.92648,
 -6.9193,
 -6.91818,
 -6.91355,
 -6.91262,
 -6.91219,
 -6.90464,
 -6.90314,
 -6.90235,
 -6.90081,
 -6.89975,
 -6.89912,
 -6.89899,
 -6.8986,
 -6.89837,
 -6.89834,
 -6.89708,
 -6.89655,
 -6.89655,
 -6.89488,
 -6.894,
 -6.89216,
 -6.88936,
 -6.88714,
 -6.88498,
 -6.88456,
 -6.8844,
 -6.8818,
 -6.87826,
 -6.87641,
 -6.87296,
 -6.87174,
 -6.87156,
 -6.87038,
 -6.86998,
 -6.86194,
 -6.85773,
 -6.84124,
 -6.83612,
 -6.83122,
 -6.83074,
 -6.83066,
 -6.83004,
 -6.82864,
 -6.82168,
 -6.815